<a href="https://colab.research.google.com/github/sajadamouei/chip-seq-dataset-decoding/blob/main/main_decode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Directory structures

!pip install seedir # Python package that provides a visual representation of directory structures in the form of tree-like structures

import seedir as sd
sd.seedir('/path/to/parent/folder', style='emoji') # replace with the actual path to your parent folder

In [ ]:
# Read and show .bedGraph files

content = []
with open("/path/to/a/bedGraph/file/coverage.bedGraph")as f:
    for line in f:
        content.append(line.strip().split())

content

In [ ]:
# Read and show .bed files

content = []
with open("/path/to/a/bed/file/labels.bed")as f:
    for line in f:
        content.append(line.strip().split())

content

In [ ]:
import os, gzip, shutil, pickle
import numpy as np


# Unpack the .gz files to obtain the .bedGraph files, then remove the original .gz files

# Input parent folder path
folder_path = '.../H3K9me3_TDH_BP/samples'

z=0
o=0
def gz_extract(directory):
    extension = ".gz"
    os.chdir(directory)
    for item in os.listdir(directory): # loop through items in dir
      if item.endswith(extension): # check for ".gz" extension
          gz_name = os.path.abspath(item) # get full path of files
          file_name = (os.path.basename(gz_name)).rsplit('.',1)[0] #get file name for file within
          with gzip.open(gz_name,"rb") as f_in, open(file_name,"wb") as f_out:
              shutil.copyfileobj(f_in, f_out)
          os.remove(gz_name) # delete zipped file


for path, subdirs, files in os.walk(folder_path):
    for name in files:
      if (z%2) == 0:
        #print(os.path.join(path))
        gz_extract(os.path.join(path))
        o+=1
      z+=1

print('Number of files in folders = ',z)
print('Number of extracted files = ', o,'\n\n')


# Read coverage files address

lst_ad = []
num=0

for root, dirs, files in os.walk(folder_path):
    for file in files:
        if file.endswith(".bedGraph"):
            file_path = os.path.join(root)
            lst_ad.append(file_path) # Add coverage files address to a list
            num+=1

print('Number of coverage files = ', num)



# Decode coverage.bedgraph files from run-length encoding

f_data = []
    
for l in range(len(lst_ad)): # Read the coverage and label files
    coverage = []
    with open(os.path.join(lst_ad[l],"coverage.bedGraph"))as f:
        for line in f:
            coverage.append(line.strip().split())

    labels = []
    with open(os.path.join(lst_ad[l],"labels.bed"))as f:
        for line in f:
            labels.append(line.strip().split())
        
    n = int(coverage[len(coverage)-1][2]) # Last genomic position
    m = int(coverage[0][1]) # First genomic position

    data = np.zeros(shape=(2,n-m)) # One array for each coverage file

    count = 0

    for i in range(len(coverage)): # Decode coverage files and move to a 2D array
        for j in range(int(coverage[i][2])-int(coverage[i][1])):
            data[0,count] = int(coverage[i][1]) + j + 1
            data[1,count] = int(coverage[i][3])
            count += 1
        
    for i in range(len(labels)): # Save labeled parts of coverage file
            
        t, s_point = np.where(data == int(labels[i][1]))
        t, e_point = np.where(data == int(labels[i][2]))
        
        # Pick up extra length from each side by 5000
        s_point = int(s_point)-5000
        e_point = int(e_point)+5000
        
        f_data.append(coverage[0][0])
        f_data.append(list(data[1,s_point:e_point]))
        f_data.append(labels[i][3])

    print('Progress of decoding= ',l+1,'/',len(lst_ad))

In [ ]:
# Save decoded dataset with label in a .ob file

with open('.../Input/a/path/Select_a_name.ob', 'wb') as fp:
    pickle.dump(f_data, fp)